In [3]:
# Carregar Bibliotecas
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [4]:
# Carregar Dados
dados = pd.read_csv('dados-gerais-das-reclamacoes-por-operadora.csv', sep=';', encoding='latin-1')
dados = pd.DataFrame(dados)
display(dados)

,Registro ANS,Razão Social,Beneficiários,Número Demanda,Data Atendimento,Classificação,Natureza Demanda,Subtema Demanda,Competência,Data de Atualização
0,326305,AMIL ASSISTÊNCIA MÉDICA INTERNACIONAL S.A.,5120557,5484808,04/01/2022 09:26,EM ANDAMENTO,Não Assistencial,Itens Obrigatórios e Cláusulas Contratuais,202208,13/09/2022 10:42
1,326305,AMIL ASSISTÊNCIA MÉDICA INTERNACIONAL S.A.,5120557,5502360,14/01/2022 18:23,EM ANDAMENTO,Não Assistencial,Mensalidade ou Contraprestação Pecuniária,202208,13/09/2022 10:42
2,302091,SÃO FRANCISCO SISTEMAS DE SAÚDE SOCIEDADE EMPR...,0,5502596,16/01/2022 12:50,EM ANDAMENTO,Não Assistencial,Mensalidade ou Contraprestação Pecuniária,202208,13/09/2022 10:42
3,326305,AMIL ASSISTÊNCIA MÉDICA INTERNACIONAL S.A.,5120557,5506224,18/01/2022 14:50,EM ANDAMENTO,Não Assistencial,Mensalidade ou Contraprestação Pecuniária,202208,13/09/2022 10:42
4,326305,AMIL ASSISTÊNCIA MÉDICA INTERNACIONAL S.A.,5120557,5509106,20/01/2022 08:09,EM ANDAMENTO,Não Assistencial,Itens Obrigatórios e Cláusulas Contratuais,202208,13/09/2022 10:42
...,...,...,...,...,...,...,...,...,...,...
252130,421154,SAÚDE BRASIL ASSISTÊNCIA MÉDICA LTDA.,53887,5804499,22/08/2022 09:42,INATIVA,Assistencial,Gerenciamento das Ações de Saúde por Parte da ...,202208,13/09/2022 10:42
252131,421154,SAÚDE BRASIL ASSISTÊNCIA MÉDICA LTDA.,53887,5806653,23/08/2022 09:07,INATIVA,Assistencial,Gerenciamento das Ações de Saúde por Parte da ...,202208,13/09/2022 10:42
252132,353353,UNIMED TERESINA - COOPERATIVA DE TRABALHO MÉDICO,110146,5806982,23/08/2022 10:32,INATIVA,Assistencial,Reembolso,202208,13/09/2022 10:42
252133,303267,UNIMED JUNDIAI - COOPERATIVA DE TRABALHO MÉDICO,97188,5808778,23/08/2022 21:19,INATIVA,Assistencial,Rol de Procedimentos e Coberturas (geográfica ...,202208,13/09/2022 10:42


In [19]:
# ETL dos Dados
etl1 = dados.count()
etl2 = dados.Classificação.value_counts(dropna=False)
display(etl1, etl2)



Registro ANS           252135
Razão Social           252135
Beneficiários          252135
Número Demanda         252135
Data Atendimento       252135
Classificação          245807
Natureza Demanda       252135
Subtema Demanda        252128
Competência            252135
Data de Atualização    252135
dtype: int64

INATIVA         186828
EM ANDAMENTO     27285
NP               15769
NÚCLEO           11869
NaN               6328
RVE               4056
Name: Classificação, dtype: int64

Registro ANS           252135
Razão Social           252135
Beneficiários          252135
Número Demanda         252135
Data Atendimento       252135
Classificação          245807
Natureza Demanda       252135
Subtema Demanda        252128
Competência            252135
Data de Atualização    252135
dtype: int64